# Monero & Ring Signatures


The Elliptic Curve Diffie Hellman Equation allows for private keys to sign information using public keys where the private key cannot be deduced from the public key. 

The problem is that you can generate only one key from this private key: can we do better?

## Curve Ed25519

The elliptic curve used in Monero is [a very specific one](https://en.wikipedia.org/wiki/Curve25519):

$$
-x^2 + y^2 = 1 - \frac{121665}{121666} x^2 y^2
$$

Alternatively written in elliptic form as:

$$
y^2 = x^3 + 486662x^2 + x
$$

Whose order is really large (76 digits, 253 bits)

$ n = 2^{252} + 27742317777372353535851937790883648493 $

and has the corresponding prime field (same as number?)

$ p = 2^{255} - 19 $

Points on Ed25519 can be expressed as 64 bytes and we can use _point compression_ to get this even lower to ~32 bytes -- won't get into this because its not entirely relevant to how Monero's ring signature magic works.



Questions:

  * Section 2.3.2 describes how to generate addresses -- this needs to be described
  * $ \mathcal{H} $ sometimes accepts two values -- there's something about how the value is hashed as the binary addition of two values that I need to verify in the early stages of the text. 
  
  
  
Answers:

  * _What is $\mathcal{H}$ in the OTA generation?_ => a cryptographically secure hash function

## Ring Signatures via Schnorr-like Signatures

### Basic Schnorr Signatures

How do we prove that somebody knows private key _k_ of a given public key _K_ without revealing any information?

_Prover_ wants to prove they have the correct private key while _Verifier_ wants to verify that _Prover_ is telling the truth.

1. Prover generates a random integer $ \alpha \in_R \mathbb{Z}_l$, computes $\alpha G$, and sends that value to the verifier. 

2. Verifier generates a random challenge $ c \in_R \mathbb{Z}_l $ and sends that to the prover. 

3. Prover computes a response $ r = \alpha = c * k $ and sends that to the verifier. 

Exchange so far -- everybody has the following:

  * $\alpha G$
  * $c$
  * $ r = \alpha = c * k $
  
4. Verifier then computes $ R = r G $ and $ R' = \alpha G + c * K $ and checks that $ R \stackrel{?}{=} R' $


__________________________

can randomly generate c for this? with `random.randint`? -- `sys.maxint`

## How Monero Works

User Keys are _two_ sets of private/public keys:

$$ (k^{v}, K^{v})\ \&\ (k^{s}, K^{s}) $$

The address of a user is their pair of public keys: $ (K^v, K^s) $

Their generation is described in section 2.3.2 of the paper -- _the Diffie-Helmann method using Elliptic cryptography_. 

These two sets of keys will allow for _function segregation_: view key and spend key. The view key will let them determine if an address owns the output and the spend key will allow them to spend that output in a transaction (or retroactively figure out how it has been spent). 


### One-Time Address

Monero's keys are used to generate one-time addresses using a DF-like exchange. 

Suppose Alice wants to send $\alpha$ to Bob (some of this may be 0 as my notes got fuzzy writing this out)

1. Alice generates a random number $ r \in_R \mathbb{Z}_l $ and calculates a one time address (OTA). That OTA is calculated as follows:

$$ K^o = \mathcal{H}_n (rK^{v}_{B})G + K_{B}^{s} $$

2. Alice then sets $ K^o $ as the addressee of the payment (for receiving), and adds the output amount $\alpha$ and the value $rG$ to the transaction data, submitting it to the network.

3. Bob receives the data. He can see $rG$ and $K^o$ -- as can everybody on the network. He then calculates two equations:

$$ k_{B}^{v}rG = rK_{B}^{v} $$

$$ K_{B}^{ls} = K^o - \mathcal{H}_n(rK_{B}^{v})G $$

He then confirms that $ K_{B}^{ls} = K_{B}^{s} $ to know that the output is addressed to him. 

The private key $ k_{B}^{v} $ is called the 'view key' because anybody who has it and the public spend key can know $ K_{B}^{ls} $ for every transaction in the output of the blockchain and thus know each transaction that belongs to Bob.

4. The one-time keys for the output are:

$$ K^o = \mathcal{H}_n(rK_{B}^v)G + K_{B}^s = (\mathcal{H}_n(rK_{B}^v) + k_{B}^s)G $$

$$ k^o = \mathcal{H}_n (rK_{B}^v) + k_{B}^s $$

For Bob to now spend this amount in a new transaction, he needs to prove ownership by signing a message with the one-time key $ K^o $. The private key $ k_{B}^s $ is the "spend key" since it is required for proving output ownership while $ k_{B}^v $ is the view key since it can be used to find the outputs spendable by Bob. 

(There's additional equations on page 39 which use output indices to ensure that generated addresses are different)


____________________________________

Do we store the random number c here? It would be public as the prover & verifier exchange it in the prior example

In [1]:
# Could the above be turned into its own notebook? -- I'd say yes


# generate two pairs of keys for alice and bob -- maybe use a data struct for this?
# also a special data struct for transaction information -- will try to query this later


# generate a OTA 


# create the transaction for this


# write query methods to look at all interactions (using Bob's public key view key) 
# and all outputs (using Bob's public output key) -- apparently you can't do this?
# then write a master query that looks at all transactions to see which are yours vs not

## Subaddresses in Monero

Bob generates his  _$i^{th}$_ subaddress using both his public keys:

$$ K^{s,i} = K^s + \mathcal{H}_n (k^v, i)G $$

$$ K^{v,i} = k^v K^{s,i} $$

Which can be alternatively written as:

$$ K^{v,i} = k^v (k^s + \mathcal{H}_n (k^v, i)) G $$

$$ K_{s,i} = (k^s + \mathcal{H}_n (k^v, i)) G $$


### Sending to a Subaddress

Let's walk through sending an amount '0':

1. Alice generates a random number and calculates a one-time addess

Random Number: 

$$ r \in_R \mathbb{Z}_l $$

OTA Generation:

$$ K^o = \mathcal{H}_n (r K_{B}^{v,1}, 0) G + K_{B}^{s,1} $$


2. Alice then sets $ K^o $ as the addressee of the payment, adds the output amount '0' and the value $ r K_{B}^{s,1} $ to the transaction data, and submits it back to the network. 

3. Bob receives this data and sees the values $ r K_{B}^{s,1} $ and $ K^o $. He can calculate two equations:

$$ k_{B}^v r K_{B}^{s,1} = r K_{B}^{v,1} $$

$$ K_{B}^{ls} = K^o - \mathcal{H}_n (rK_B^{v,1}, 0) G $$

He can verify if the transaction if addressed to him by ensuring that:

$$ K_{B}^{1s} = K_{B}^{s,1} $$

Bob only needs his private view key _$ k_B^v $_ and public spend key _$ K_B^{s,1} $_ to find transaction outputs send to his subaddress. 

4. the one-time keys for the output are:

$$ K^o = \mathcal{H}_n (r K_B^{v,1}, 0)G + K_B^{s,1} = (\mathcal{H}_n (r K_B^{v,1}, 0) + k_B^{s,1}) G $$

$$ k^o = \mathcal{H}_n (r K_B^{v,1}, 0) + k_B^{s,1}$$

# How does Monero hide amounts?

Monero uses _committments_ to hide output amounts from everyone except the sender and receiver. 

(on Pedersen Committments on page 44)